In [2]:
import pandas as pd
import sqlite3 as sql
from pathlib import Path
import mysql.connector
#from mysql.connector import connect, Error
#from sqlalchemy import create_engine
import os

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
pd.set_option('display.max_rows',200)
pd.set_option('display.max_rows',200)

In [5]:
path='/home/dpv/Desktop/olist-marketing/conf/.env'
if (os.environ.get("DB_TYPE") == None):
    from dotenv import load_dotenv
    load_dotenv(path)


In [6]:
PASSWORD=os.environ.get("MYSQL_PWD")
USER=os.environ.get("MYSQL_USER")


In [7]:
config = {
  'user': USER,
  'password': PASSWORD,
  'host': '127.0.0.1',
  'database': 'olist',
  'raise_on_warnings': True
}

db_conn = mysql.connector.connect(**config)

#db_conn.close()

In [8]:
def get_connection():
  config = {
    'user': USER,
    'password': PASSWORD,
    'host': '127.0.0.1',
    'database': 'olist',
    'raise_on_warnings': True
  }

  db_conn = mysql.connector.connect(**config)
  return db_conn

#db_conn.close()

In [9]:
path='/home/dpv/Desktop/olist-marketing/data/' #path to folder

In [10]:
def exec_query(query):
    print("Starting querry...")
    db_conn=get_connection()
    cursor = db_conn.cursor(buffered=True)
    cursor.execute(query)
    print('Query executed')
    cursor.close() 
    #cursor = db_conn.cursor(buffered=True)



In [11]:
mycursor = db_conn.cursor(buffered=True)
 
mycursor.execute("Show tables;")
 
myresult = mycursor.fetchall()
 
for x in myresult:
    print(x)

('cust_prod_ordr_payt',)
('customer_orders',)
('customer_products_orders',)
('customers',)
('customers_geo',)
('customers_merged',)
('geolocation',)
('order_items',)
('order_payments',)
('order_reviews',)
('orders',)
('orders_and_customers',)
('orders_and_items',)
('products',)
('products_eng',)
('sellers',)
('translations',)


In [8]:
mycursor.execute("SELECT COUNT(*) FROM customers;")
myresult = mycursor.fetchall()
for x in myresult:
    print(x)

(99441,)


In [42]:
customers=pd.read_sql(
  """
        select *
        from customers
        
  """, db_conn)
geolocation=pd.read_sql(
  """
        select *
        from geolocation
        
  """, db_conn)
order_items=pd.read_sql(
  """
        select *
        from order_items
        
  """, db_conn)

order_payments=pd.read_sql(
  """
        select *
        from order_payments
        
  """, db_conn)

order_reviews=pd.read_sql(
  """
        select *
        from order_reviews
        
  """, db_conn)


orders=pd.read_sql(
  """
        select *
        from orders
        
  """, db_conn)


products=pd.read_sql(
  """
        select *
        from products
        
  """, db_conn)


sellers=pd.read_sql(
  """
        select *
        from sellers
        
  """, db_conn)


translations=pd.read_sql(
  """
        select *
        from translations
        
  """, db_conn)


tables = [customers, geolocation, order_items, order_payments,order_reviews,
orders,products,sellers,translations]



In [10]:
for table in tables:
    print(table.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  object 
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-n

In [7]:
pd.read_sql(
  """
        SHOW TABLES;

  """, db_conn)

,Tables_in_olist
0,customer_orders
1,customer_products_orders
2,customers
3,customers_geo
4,geolocation
5,order_items
6,order_payments
7,order_reviews
8,orders
9,orders_and_customers


### Join Customers and Geolocations

In [27]:
# query= pd.read_sql(
#   """
#         SELECT 
#         customers.* ,
#         geolocation.geolocation_lat, geolocation.geolocation_lng
#         FROM customers 
#         LEFT JOIN geolocation 
#         ON customers.customer_zip_code_prefix=geolocation.geolocation_zip_code_prefix
#         ;

#   """, db_conn)

# customers_geo = pd.DataFrame(query)

In [ ]:
# def createTable(query):
#     cursor = db_conn.cursor()
#     table= query
#     cursor.execute(table)
#     print(f'Table {table} created') 
# createTable(query=query_)

In [12]:
# query="DROP TABLE IF EXISTS customers_geo"
# exec_query(query)

Query executed


In [19]:
# query="DROP TABLE IF EXISTS orders_and_customers;"
# exec_query(query)

In [13]:
query = ("CREATE TABLE customers_geo AS (SELECT customers.* ,geolocation.geolocation_lat, geolocation.geolocation_lng FROM customers LEFT JOIN geolocation  ON customers.customer_zip_code_prefix=geolocation.geolocation_zip_code_prefix);")
exec_query(query)
#createTable(query=query_)

In [12]:
# query_geo= pd.read_sql(
#   """
#         CREATE TEMPORARY TABLE customer_geo_temp
#         SELECT 
#         customers.* ,
#         geolocation.geolocation_lat, geolocation.geolocation_lng
#         FROM customers 
#         LEFT JOIN geolocation  
#         ON customers.customer_zip_code_prefix=geolocation.geolocation_zip_code_prefix
#         ;

#   """, db_conn)


In [44]:
customers_geo=pd.read_sql(
  """
        select *
        from customers_geo
        
  """, db_conn)

In [45]:
customers_geo.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_lat,geolocation_lng
0,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,-23.531294,-46.656404
1,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,-23.533334,-46.654074
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,-23.530159,-46.657632
3,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,-23.531294,-46.656404
4,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,-23.530118,-46.657771


### Join orders with order items, then with customers_geo

In [34]:
query= pd.read_sql(
  """
        SELECT 
        orders.* ,
        order_items.seller_id, order_items.shipping_limit_date, order_items.price, order_items.freight_value
        FROM orders 
        LEFT JOIN order_items  
        ON orders.order_id=order_items.order_id
        ;

  """, db_conn)

orders_ = pd.DataFrame(query)

In [13]:
query=("""
        DROP TABLE IF EXISTS orders_and_items;
        CREATE TABLE orders_and_items
        SELECT 
        orders.* ,
        order_items.seller_id, order_items.shipping_limit_date, order_items.price, order_items.freight_value,
        order_items.order_item_id, order_items.product_id
        FROM orders 
        LEFT JOIN order_items  
        ON orders.order_id=order_items.order_id
        ;
  """)
exec_query(query)

Starting querry...
Query executed


In [24]:
query=("""
        CREATE TABLE orders_and_customers
        SELECT 
        customers_geo.* ,
        orders_and_items.order_id, 
        orders_and_items.order_status,
        orders_and_items.order_purchase_timestamp,
        orders_and_items.order_approved_at,
        orders_and_items.order_delivered_carrier_date,
        orders_and_items.order_delivered_customer_date,
        orders_and_items.order_estimated_delivery_date,
        orders_and_items.shipping_limit_date,
        orders_and_items.seller_id,
        orders_and_items.price,
        orders_and_items.freight_value
        FROM customers_geo
        LEFT JOIN orders_and_items   
        ON customers_geo.customer_id=orders_and_items.customer_id
        ;
  """)
exec_query(query)

Query executed


In [26]:
orders_and_customers=pd.read_sql(
  """
        select *
        from orders_and_customers
        
  """, db_conn)
orders_and_customers.to_csv(path+'orders_and_customers.csv')

In [17]:
query= pd.read_sql(
  """
        SELECT 
          customers.customer_unique_id, 
          orders.order_purchase_timestamp,
          order_payments.payment_value

        FROM customers 
        JOIN orders  ON orders.customer_id = customers.customer_id 
        JOIN order_payments on orders.order_id = order_payments.order_id
        -- WHERE NOT orders.order_status= 'canceled'       


  """, db_conn)

df = pd.DataFrame(query)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 3 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   customer_unique_id        103886 non-null  object 
 1   order_purchase_timestamp  103886 non-null  object 
 2   payment_value             103886 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.4+ MB


In [13]:
df.head()

,customer_unique_id,order_purchase_timestamp,payment_value
0,bf635aadfff238fd7e9b0d25e800d019,2017-09-25 16:38:46,26.58
1,55020cb68ae358a51969695657a54e9c,2018-06-11 14:59:24,340.08
2,d02cbb85434c84acabb8e81bbf064d3e,2018-01-14 18:49:14,48.91
3,cb409daad1fb6069e979463ca3de23dc,2017-07-18 13:32:25,154.14
4,23daf66dd86071f57d332fa5562afbe9,2018-05-23 16:37:35,20.75


In [18]:
#full_path=Path('/var/lib/mysql/')
#full_path.exists()

### Get translations

In [49]:
# Running into following error, done directly in Workbench
# ProgrammingError: 1054 (42S22): Unknown column 'translations.product_category_name' in 'on clause'

# query=("""
#         CREATE TABLE products_eng
#         SELECT products.product_id, translations.product_category_name_english,products.product_photos_qty,products.product_weight_g,products.product_length_cm, 
#         products.product_height_cm, products.product_width_cm
#         FROM products
#         LEFT JOIN translations   
#         ON products.product_category_name=translations.product_category_name
#         ;
#   """)
# exec_query(query)

In [51]:

products_eng=pd.read_sql(
  """
        select *
        from products_eng;
        
  """, db_conn)
products_eng.to_csv(path+'products_eng.csv')

In [ ]:
query=("""
        CREATE TABLE customer_orders
        SELECT customers_geo.*,ordrers.
        FROM customers_geo
        LEFT JOIN orders   
        ON products.product_category_name=translations.product_category_name
        ;
  """)
exec_query(query)

In [26]:
# query=("""
# CREATE   TABLE customer_orders
#         SELECT customers_geo.customer_unique_id,
#         customers_geo.customer_zip_code_prefix,
#         customers_geo.customer_city, customers_geo.customer_state,
#         customers_geo.geolocation_lat,
#         customers_geo.geolocation_lng,
#         orders_and_items.order_id,
#         orders_and_items.order_purchase_timestamp,
#         orders_and_items.order_item_id,
#         orders_and_items.product_id, orders_and_items.price,
#         orders_and_items.freight_value
#         FROM customers_geo
#         LEFT JOIN orders_and_items
#         ON orders_and_items.customer_id=customers_geo.customer_id
#         ;
#   """)

# exec_query(query)


In [18]:
query=("""

CREATE TABLE customer_product_orders
        SELECT customer_orders.*,
        products_eng.product_category_name_english,products_eng.product_photos_qty,product_weight_g,product_length_cm,product_height_cm,
        product_width_cm
        FROM customer_orders
        LEFT JOIN products_eng
        ON customer_orders.product_id=products_eng.product_id
        ;
  """)

exec_query(query)

Starting querry...
Query executed


In [22]:
pd.read_sql(
  """
        SELECT DISTINCT * FROM customer_product_orders
         LIMIT 10;

  """, db_conn)

DatabaseError: 1412 (HY000): Table definition has changed, please retry transaction

In [29]:
# query=("""
# DROP TABLE IF EXISTS customer_products_orders;
#   """)

# exec_query(query)

In [28]:
#Introduced duplicates, cleaning up
query=("""

CREATE TABLE customer_products_orders
        SELECT DISTINCT * FROM customer_product_orders
        ;
DROP TABLE customer_product_orders;
  """)

exec_query(query)

Starting querry...
Query executed


In [10]:
# db_conn = mysql.connector.connect(**config)

# customer_products_orders=pd.read_sql(
#   """
#         select *
#         from customer_products_orders;
        
#   """, db_conn)


: 

: 

In [ ]:
customer_products_orders.to_csv(path+'/joined-data/cstmr_prdct_ordrs.csv')

In [12]:
pd.read_sql(
  """
        SELECT DISTINCT * FROM customer_products_orders
         LIMIT 10;

  """, db_conn)

,customer_unique_id,order_id,order_purchase_timestamp,order_item_id,product_id,price,freight_value,product_category_name_english,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,0ee84527dd65b0a7fadd79df2fc273fe,0070092bb6004faafa287e5fc35971e9,2017-07-25 11:42:58,1,31a2f42a87890f87d77daebdfabc182e,149.00,14.06,toys,5,1700,30,35,25
1,095ffc730d9d2da2f12a0bec18722aae,00404fa7a687c8c44ca69d42695aae73,2018-05-07 22:38:21,1,53b36df67ebb7c41585e8d54d6772e08,99.90,0.00,watches_gifts,3,584,16,11,13
2,051be26c01eaee2404f19132b80da15f,0068468c453d28c8ef3fd089e50a5847,2017-11-24 22:25:25,1,2f763ba79d9cd987b2034aac7ceffe06,469.00,20.85,electronics,2,595,8,6,6
3,a8d771b4e13abd8d9df3b4f6c0a38091,01e9b3d727e46edf036b0bd549c38bb4,2018-01-20 21:14:48,1,af56e0203193f066ed64cbbbbf95ae6a,99.90,16.46,sports_leisure,3,750,25,7,25
4,747f419f11717e7a52d1175315a2365b,017dcb6892d4469a13dee34f241f8c4e,2017-11-12 20:17:40,1,f4d705aa95ccca448e5b0deb6e5290ba,24.90,9.09,bed_bath_table,1,250,20,2,15
5,26ffd81154d2a5dbe5cae3dce93289bd,00c4f17dcb94448f27a2cbaa83f10a4b,2018-06-08 19:18:56,1,adf591c625cb265c12bc6749d3a2f757,32.00,7.39,health_beauty,6,150,20,24,33
6,3e13e5843eacac8a8ec14b42e8a7174c,00b47ee856118583dbec7d1f9b18d847,2018-03-26 19:58:48,1,0e40b1ed4cfd3da1962ec91913e54ba8,13.65,15.23,electronics,1,175,22,11,15
7,4b5c728e4fd50ebdfb7799ff5754eee0,0207f5a2635f4e2d29acd00fc01a36ba,2018-03-05 10:25:28,1,6b017e0d95d8765a6f2bed3bc272c61e,59.90,11.92,fashion_underwear_beach,1,250,36,4,20
8,2c078ecf4c8eaa632660d66f124be60d,0203892ed7a2b76401830f4b82a5a34c,2017-07-08 20:26:08,1,c6336fa91fbd87c359e44f5dca5a90ed,39.90,8.72,sports_leisure,1,150,16,16,11
9,07187f9425311d0bd2a5451169bb8c3c,032a25a524b766bdb806500361630bea,2018-07-11 17:59:45,1,782447ddd6d2ba5b64cb23ef7a8db626,48.98,23.10,home_appliances_2,1,3585,33,6,19


In [25]:
feats=list(customer_products_orders.columns)
feats_=['customer_products_orders.'+feat for feat in feats]




In [26]:
feats

['customer_unique_id',
 'order_id',
 'order_purchase_timestamp',
 'order_item_id',
 'product_id',
 'price',
 'freight_value',
 'product_category_name_english',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm']

In [27]:
string = ''
for feat in feats:
    string+=','+feat
string

',customer_unique_id,order_id,order_purchase_timestamp,order_item_id,product_id,price,freight_value,product_category_name_english,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm'

In [ ]:
query=("""

CREATE TABLE cust_prod_ordrs_full
        SELECT customer_products_orders.*,
        customer_products_orders.customer_unique_id,
        customer_products_orders.order_id,
        customer_products_orders.order_purchase_timestamp,

        FROM customer_products_orders
        LEFT JOIN products_eng
        ON customer_orders.product_id=products_eng.product_id
        ;
  """)

exec_query(query)

In [32]:
pd.read_sql(
  """
        SELECT  
            customer_unique_id,product_id,
            MAX(order_purchase_timestamp),
            SUM(price),
            SUM(freight_value),
            SUM(product_photos_qty),
            SUM(product_weight_g)    
        FROM customer_products_orders
        GROUP BY 1,2;

  """, db_conn)

,customer_unique_id,product_id,MAX(order_purchase_timestamp),SUM(price),SUM(freight_value),SUM(product_photos_qty),SUM(product_weight_g)
0,0ee84527dd65b0a7fadd79df2fc273fe,31a2f42a87890f87d77daebdfabc182e,2017-07-25 11:42:58,149.00,14.06,5.0,1700.0
1,095ffc730d9d2da2f12a0bec18722aae,53b36df67ebb7c41585e8d54d6772e08,2018-05-07 22:38:21,99.90,0.00,3.0,584.0
2,051be26c01eaee2404f19132b80da15f,2f763ba79d9cd987b2034aac7ceffe06,2017-11-24 22:25:25,469.00,20.85,2.0,595.0
3,a8d771b4e13abd8d9df3b4f6c0a38091,af56e0203193f066ed64cbbbbf95ae6a,2018-01-20 21:14:48,99.90,16.46,3.0,750.0
4,747f419f11717e7a52d1175315a2365b,f4d705aa95ccca448e5b0deb6e5290ba,2017-11-12 20:17:40,24.90,9.09,1.0,250.0
...,...,...,...,...,...,...,...
102743,49836f57f840f0c1eb0b5cb343312df7,5e53d7f0369429a1b28ef093a15f7509,2017-09-02 22:27:41,689.90,40.15,3.0,900.0
102744,3ee793b025a9f8aff7bc7b3236b97e9f,d77d9f61eda0118690a342a000144563,2018-01-27 15:24:10,129.07,16.66,2.0,400.0
102745,b3dab5cb596c07eaf96a4325b61f6896,75d6b6963340c6063f7f4cfcccfe6a30,2017-06-23 09:08:13,56.99,14.15,2.0,400.0
102746,d8b67ce6d6c062269add92dd22e8cc94,bf7727705a701b9c135cb00df5a2dc5d,2017-07-09 09:18:56,49.90,19.59,2.0,1500.0


In [ ]:
# Decide not to go down the route below due to information loss
# pd.read_sql(
#   """
#         SELECT  
#             customer_unique_id,product_id,
#             MAX(order_purchase_timestamp),
#             SUM(price),
#             SUM(freight_value),
#             SUM(product_photos_qty),
#             SUM(product_weight_g)    
#         FROM customer_products_orders
#         GROUP BY 1,2;

#   """, db_conn)

In [35]:
pd.read_sql(
  """
        SELECT  
            order_id,
            SUM(payment_installments) as payment_installments,
            SUM(payment_value) as payment_value 
        FROM order_payments
        GROUP BY 1;

  """, db_conn)

,order_id,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,8.0,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1.0,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1.0,65.71
3,ba78997921bbcdc1373bb41e913ab953,8.0,107.78
4,42fdf880ba16b47b59251dd489d4441a,2.0,128.45
...,...,...,...
99435,0406037ad97740d563a178ecc7a2075c,1.0,363.31
99436,7b905861d7c825891d6347454ea7863f,2.0,96.80
99437,32609bbb3dd69b3c066a6860554a77bf,1.0,47.77
99438,b8b61059626efa996a60be9bb9320e10,5.0,369.54


In [36]:
query=("""

CREATE TEMPORARY TABLE payment_orders
        SELECT  
            order_id,
            SUM(payment_installments) as payment_installments,
            SUM(payment_value) as payment_value 
        FROM order_payments
        GROUP BY 1;
  """)

exec_query(query)

Starting querry...
Query executed


In [38]:
query=("""

CREATE TABLE cust_prod_ordr_payt
        SELECT customer_products_orders.*,
        payment_orders.payment_installments,
        payment_orders.payment_value
        FROM customer_products_orders
        LEFT JOIN payment_orders
        ON customer_products_orders.order_id=payment_orders.order_id
        ;
  """)

exec_query(query)

Starting querry...
Query executed


In [39]:
pd.read_sql(
  """
        SELECT  
            *
        FROM cust_prod_ordr_payt
        LIMIT 10;

  """, db_conn)

,customer_unique_id,order_id,order_purchase_timestamp,order_item_id,product_id,price,freight_value,product_category_name_english,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_installments,payment_value
0,051be26c01eaee2404f19132b80da15f,0068468c453d28c8ef3fd089e50a5847,2017-11-24 22:25:25,1,2f763ba79d9cd987b2034aac7ceffe06,469.00,20.85,electronics,2,595,8,6,6,3.0,489.85
1,96a9e66285108791875512fa49cab22f,016befe103ce7c9e5b1b7ef359777eea,2017-09-03 10:11:52,1,6c321ed0f47858a8650f4104771224b5,49.00,11.85,watches_gifts,4,300,16,2,20,3.0,60.85
2,803095e5d7c0d6fb29faadfbe65b8e53,031a7dab65c3430d325ad35ab8a95277,2018-06-21 22:17:19,1,18fa9cc25ea8b54f32d029f261673c0f,109.90,18.82,construction_tools_construction,1,2900,23,8,23,6.0,128.72
3,2595282f2e08ccd2b80be10dad0fa46f,250eede6202cff2981de57cba5c1b08c,2017-04-05 12:07:47,1,e5433d154f98bcb2c139b9679fce0814,48.90,14.52,perfumery,2,250,22,5,15,3.0,63.42
4,8fd4869508146c4f423b0860ef33d583,27dfe580597e36bfdb283184aa96a201,2017-06-19 19:45:31,1,4a96de21ccfb22ebe1c56992acbbd7fc,99.00,17.94,air_conditioning,5,1300,30,10,30,1.0,116.94
5,a307cb0934b15ffb99d5247fb5616107,82ee69216f11b23e2d21d2796e079366,2018-08-08 11:34:56,1,a04087ab6a96ffa041f8a2701a72b616,650.00,42.94,computers,1,13700,27,50,53,4.0,692.94
6,7361b4cce726b898deb902ee3fe27f28,2e1f1b86124165634a36f9954e368b49,2018-01-14 19:16:48,1,1e618d311a1b7f88a9d96ec50aa85582,129.00,18.52,housewares,4,9950,31,61,31,1.0,147.52
7,73c4efed8d45dab6e5e412c2f3594fe6,048e6e4623dbf118c43e0f5572016faa,2017-12-31 17:57:21,1,a04087ab6a96ffa041f8a2701a72b616,599.00,58.63,computers,1,13700,27,50,53,4.0,657.63
8,d29cd7ea1f16713ca6ca68c50bb2039d,51fb61776ef5371653e8b4e1c99d1c12,2017-04-28 12:40:16,1,38b5079a807bb9f22ddfa46f929accee,21.00,10.96,perfumery,5,200,16,2,11,3.0,31.96
9,a9a54fee4e5566e8070dd4029e1ccf22,984cefe3260707e1f6fc38b18b2b276c,2017-05-08 13:45:25,1,f6bd73f359ad6b91076b19cc0b70b44a,74.99,27.06,computers_accessories,5,1050,29,16,18,4.0,102.05


In [11]:
db_conn = mysql.connector.connect(**config)

cust_prod_ordr_payt=pd.read_sql(
  """
        SELECT *
        FROM cust_prod_ordr_payt;
        
  """, db_conn)


In [14]:
joined_path = '/home/dpv/Desktop/olist-marketing/joined-data/'


In [ ]:
cust_prod_ordr_payt.to_csv(joined_path+'cust_prod_ordr_payt.csv')

In [13]:
db_conn = mysql.connector.connect(**config)

cust_prod_ordr_payt_geo=pd.read_sql(
  """
        SELECT 
        cust_prod_ordr_payt.*, 
        customers_geo.customer_city,
        customers_geo.customer_state,
        customers_geo.geolocation_lat,
        customers_geo.geolocation_lng
        FROM cust_prod_ordr_payt
        LEFT JOIN customers_geo
        ON customers_geo.customer_unique_id=cust_prod_ordr_payt.customer_unique_id;
        
  """, db_conn)

: 

: 

### Switching over to working in Pandas for column encoding flexibility and other benefits


In [15]:
cust_prod_ordr_payt = pd.read_csv(joined_path+'cust_prod_ordr_payt.csv')


In [15]:
cust_prod_ordr_payt.head()

,customer_unique_id,order_id,order_purchase_timestamp,order_item_id,product_id,price,freight_value,product_category_name_english,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_installments,payment_value
0,051be26c01eaee2404f19132b80da15f,0068468c453d28c8ef3fd089e50a5847,2017-11-24 22:25:25,1.0,2f763ba79d9cd987b2034aac7ceffe06,469.0,20.85,electronics,2.0,595.0,8.0,6.0,6.0,3.0,489.85
1,96a9e66285108791875512fa49cab22f,016befe103ce7c9e5b1b7ef359777eea,2017-09-03 10:11:52,1.0,6c321ed0f47858a8650f4104771224b5,49.0,11.85,watches_gifts,4.0,300.0,16.0,2.0,20.0,3.0,60.85
2,803095e5d7c0d6fb29faadfbe65b8e53,031a7dab65c3430d325ad35ab8a95277,2018-06-21 22:17:19,1.0,18fa9cc25ea8b54f32d029f261673c0f,109.9,18.82,construction_tools_construction,1.0,2900.0,23.0,8.0,23.0,6.0,128.72
3,2595282f2e08ccd2b80be10dad0fa46f,250eede6202cff2981de57cba5c1b08c,2017-04-05 12:07:47,1.0,e5433d154f98bcb2c139b9679fce0814,48.9,14.52,perfumery,2.0,250.0,22.0,5.0,15.0,3.0,63.42
4,8fd4869508146c4f423b0860ef33d583,27dfe580597e36bfdb283184aa96a201,2017-06-19 19:45:31,1.0,4a96de21ccfb22ebe1c56992acbbd7fc,99.0,17.94,air_conditioning,5.0,1300.0,30.0,10.0,30.0,1.0,116.94


In [16]:
from sklearn.preprocessing import LabelEncoder

In [20]:
#Keep both category name and its encoding for ease of interpretation. Remember to drop product_category_name_english before model fitting
le=LabelEncoder()
cust_prod_ordr_payt['product_cat_name_le']=le.fit_transform(cust_prod_ordr_payt['product_category_name_english'])

In [21]:
cust_prod_ordr_payt.head()

,Unnamed: 0,customer_unique_id,order_id,order_purchase_timestamp,order_item_id,product_id,price,freight_value,product_category_name_english,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_installments,payment_value,product_cat_name_le
0,0,051be26c01eaee2404f19132b80da15f,0068468c453d28c8ef3fd089e50a5847,2017-11-24 22:25:25,1.0,2f763ba79d9cd987b2034aac7ceffe06,469.0,20.85,electronics,2.0,595.0,8.0,6.0,6.0,3.0,489.85,26
1,1,96a9e66285108791875512fa49cab22f,016befe103ce7c9e5b1b7ef359777eea,2017-09-03 10:11:52,1.0,6c321ed0f47858a8650f4104771224b5,49.0,11.85,watches_gifts,4.0,300.0,16.0,2.0,20.0,3.0,60.85,70
2,2,803095e5d7c0d6fb29faadfbe65b8e53,031a7dab65c3430d325ad35ab8a95277,2018-06-21 22:17:19,1.0,18fa9cc25ea8b54f32d029f261673c0f,109.9,18.82,construction_tools_construction,1.0,2900.0,23.0,8.0,23.0,6.0,128.72,17
3,3,2595282f2e08ccd2b80be10dad0fa46f,250eede6202cff2981de57cba5c1b08c,2017-04-05 12:07:47,1.0,e5433d154f98bcb2c139b9679fce0814,48.9,14.52,perfumery,2.0,250.0,22.0,5.0,15.0,3.0,63.42,59
4,4,8fd4869508146c4f423b0860ef33d583,27dfe580597e36bfdb283184aa96a201,2017-06-19 19:45:31,1.0,4a96de21ccfb22ebe1c56992acbbd7fc,99.0,17.94,air_conditioning,5.0,1300.0,30.0,10.0,30.0,1.0,116.94,1


In [22]:
product_cats = dict(zip(le.classes_,range(len(le.classes_))))

In [23]:
product_cats

{'agro_industry_and_commerce': 0,
 'air_conditioning': 1,
 'art': 2,
 'arts_and_craftmanship': 3,
 'audio': 4,
 'auto': 5,
 'baby': 6,
 'bed_bath_table': 7,
 'books_general_interest': 8,
 'books_imported': 9,
 'books_technical': 10,
 'cds_dvds_musicals': 11,
 'christmas_supplies': 12,
 'cine_photo': 13,
 'computers': 14,
 'computers_accessories': 15,
 'consoles_games': 16,
 'construction_tools_construction': 17,
 'construction_tools_lights': 18,
 'construction_tools_safety': 19,
 'cool_stuff': 20,
 'costruction_tools_garden': 21,
 'costruction_tools_tools': 22,
 'diapers_and_hygiene': 23,
 'drinks': 24,
 'dvds_blu_ray': 25,
 'electronics': 26,
 'fashio_female_clothing': 27,
 'fashion_bags_accessories': 28,
 'fashion_childrens_clothes': 29,
 'fashion_male_clothing': 30,
 'fashion_shoes': 31,
 'fashion_sport': 32,
 'fashion_underwear_beach': 33,
 'fixed_telephony': 34,
 'flowers': 35,
 'food': 36,
 'food_drink': 37,
 'furniture_bedroom': 38,
 'furniture_decor': 39,
 'furniture_living_roo

In [28]:
# f = open(joined_path+"product_cats_dictionary.txt","w")

# # write file
# f.write( str(product_cats) )

# # close file
# f.close()

In [30]:
#Better to save the dictionary to JSON
import json

# python dictionary with key value pairs

# create json object from dictionary
json_ = json.dumps(product_cats)

# open file for writing, "w" 
f = open(joined_path+"product_cats_dictionary.json","w")

# write json object to file
f.write(json_)

# close file
f.close()

In [40]:
cust_prod_ordr_payt=cust_prod_ordr_payt.sort_values(by=['customer_unique_id','order_purchase_timestamp'])
cust_prod_ordr_payt.to_csv(joined_path+'cust_prod_ordr_payt.csv')

In [16]:
cust_prod_ordr_payt.nunique()

Unnamed: 0.1                     113425
Unnamed: 0                       113425
customer_unique_id                96096
order_id                          99441
order_purchase_timestamp          98875
order_item_id                        21
product_id                        32951
price                              5968
freight_value                      6999
product_category_name_english        71
product_photos_qty                   19
product_weight_g                   2202
product_length_cm                    99
product_height_cm                   102
product_width_cm                     95
payment_installments                 28
payment_value                     27746
product_cat_name_le                  72
dtype: int64

In [46]:
cust_prod_ordr_payt_geo=pd.merge(cust_prod_ordr_payt,customers_geo,how='left', left_on='customer_unique_id',right_on='customer_unique_id')

: 

: 

In [19]:
string=""
for col in cust_prod_ordr_payt.columns:
    string += 'cust_prod_ordr_payt.'+col+','
string

'cust_prod_ordr_payt.Unnamed: 0.1,cust_prod_ordr_payt.Unnamed: 0,cust_prod_ordr_payt.customer_unique_id,cust_prod_ordr_payt.order_id,cust_prod_ordr_payt.order_purchase_timestamp,cust_prod_ordr_payt.order_item_id,cust_prod_ordr_payt.product_id,cust_prod_ordr_payt.price,cust_prod_ordr_payt.freight_value,cust_prod_ordr_payt.product_category_name_english,cust_prod_ordr_payt.product_photos_qty,cust_prod_ordr_payt.product_weight_g,cust_prod_ordr_payt.product_length_cm,cust_prod_ordr_payt.product_height_cm,cust_prod_ordr_payt.product_width_cm,cust_prod_ordr_payt.payment_installments,cust_prod_ordr_payt.payment_value,cust_prod_ordr_payt.product_cat_name_le,'